<a href="https://colab.research.google.com/github/YeomSuWoong/SOCAR-AI-BOOT-CAMP/blob/main/Assignment_1_ETA_%EC%98%88%EC%B8%A1_%EB%AA%A8%EB%8D%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [15]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
from sklearn.metrics import mean_squared_error
from sklearn import ensemble
# 이 실험적 기능을 명시적으로 필요
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import PredefinedSplit

/usr/local/lib/python3.7/dist-packages/sklearn/experimental/enable_hist_gradient_boosting.py:17: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  "Since version 1.0, "


In [16]:
tada_eta = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Chapter2/tada_eta.xlsx')
print(tada_eta.columns)
tada_eta.head()

Index(['ATA', 'id', 'api_eta', 'created_at_kst', 'month', 'pickup_lng',
       'pickup_lat', 'pickup_gu', 'driver_id', 'driver_lng', 'driver_lat',
       'hour'],
      dtype='object')


,ATA,id,api_eta,created_at_kst,month,pickup_lng,pickup_lat,pickup_gu,driver_id,driver_lng,driver_lat,hour
0,10.60,R4L9FL6NVEL4D0,5.65,2019-12-01T00:55:09,12,127.091399,37.624545,노원구,DST52944,127.069978,37.616472,0
1,9.20,R3A7JBY7CS6TPD,8.60,2019-12-01T00:09:39,12,127.023351,37.625564,강북구,DPC72995,127.026479,37.604711,0
2,10.85,R2BIUY1TDK14XE,10.53,2019-12-15T00:18:42,12,126.881154,37.481641,금천구,DPP94923,126.906225,37.452167,0
3,5.52,R31PU5GNHSGU6C,1.92,2019-07-31T00:24:11,7,126.988375,37.567897,중구,DCT73585,126.991170,37.566416,0
4,8.60,R41WR6N1VT42DJ,7.10,2019-07-05T00:40:34,7,126.988684,37.566716,중구,DPJ86237,127.006860,37.571164,0


In [17]:
enc = sklearn.preprocessing.OrdinalEncoder(dtype=np.int32)
ordinal = enc.fit_transform(np.array(tada_eta['pickup_gu']).reshape(-1,1))
tada_eta['pickup_gu'] = ordinal[:,0]
tada_eta['distance'] = ((tada_eta['pickup_lat']-tada_eta['driver_lat'])**2 + (tada_eta['pickup_lng']-tada_eta['driver_lng'])**2)*100000
tada_eta = tada_eta.drop(['id', 'created_at_kst', 'driver_id', 'pickup_lng','pickup_lat', 'driver_lng','driver_lat'],1)
tada_eta.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """


,ATA,api_eta,month,pickup_gu,hour,distance
0,10.60,5.65,12,8,0,52.400202
1,9.20,8.60,12,2,0,44.465889
2,10.85,10.53,12,7,0,149.731535
3,5.52,1.92,7,23,0,1.000535
4,8.60,7.10,7,23,0,35.014797


In [18]:
tada_eta = tada_eta.sample(frac=1, random_state=0).reset_index(drop=True)
train = tada_eta[:12000]
test = tada_eta[12000:]

In [19]:
tada_eta.head()
tada_eta.shape

(13916, 6)

In [20]:
x_train = np.array(train.drop('ATA',1))
y_train = np.array(train['ATA'])
x_test = np.array(test.drop('ATA',1))
y_test = np.array(test['ATA'])
eta_features = [x for i,x in enumerate(tada_eta.columns) if i!=0]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


In [21]:
X = np.concatenate((x_train,x_test), axis = 0)
y = np.concatenate((y_train, y_test), axis = 0)

In [22]:
pds = PredefinedSplit(test_fold=[-1]*len(x_train) + [0]*len(x_test))

In [23]:
param_grid = {'max_iter':[100, 200, 300],
             'max_depth': [2,3,4,5],
             'min_samples_leaf': [1, 20, 100],
             'learning_rate': [0.01, 0.02, 0.05],
             'loss': ['least_squares']}

In [24]:
grid_search = GridSearchCV(estimator = ensemble.HistGradientBoostingRegressor(),
                          param_grid = param_grid,
                          cv = pds, n_jobs = -1, scoring = 'neg_mean_squared_error', verbose = 2)

In [25]:
grid_search.fit(X,y)
print(grid_search.best_params_)

Fitting 1 folds for each of 108 candidates, totalling 108 fits


/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_hist_gradient_boosting/gradient_boosting.py:1306: FutureWarning: The loss 'least_squares' was deprecated in v1.0 and will be removed in version 1.2. Use 'squared_error' which is equivalent.
  FutureWarning,


{'learning_rate': 0.05, 'loss': 'least_squares', 'max_depth': 4, 'max_iter': 100, 'min_samples_leaf': 100}


In [36]:
params = {'max_iter':100,
         'max_depth':4,
         'min_samples_leaf':100,
         'learning_rate':0.05,
         'loss':'least_squares'}

In [37]:
reg = ensemble.HistGradientBoostingRegressor(**params)
reg.fit(x_train, y_train)

mse = mean_squared_error(y_test, reg.predict(x_test))
print("The mean squared error (MSE) on test set: {:.4f}".format(mse))
print("The initial error of API ETA on test set: {:.4f}".format(mean_squared_error(y_test, x_test[:,0]) ))

/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_hist_gradient_boosting/gradient_boosting.py:1306: FutureWarning: The loss 'least_squares' was deprecated in v1.0 and will be removed in version 1.2. Use 'squared_error' which is equivalent.
  FutureWarning,


The mean squared error (MSE) on test set: 8.2309
The initial error of API ETA on test set: 13.7135
